In [ ]:
# conda env list

# conda create  -n [이름] python=[버전]
# conda activate [이름]

# conda deactivate
# conda remove -n [이름] --all

In [ ]:
# 입력-> 토큰화 및 시퀀스 변화 -> 패딩(고정길이화) -> 임베딩(단어->벡터화)-> 1D Convolution + poling 반복
# -> Flatten ->Dense(은닉) -> 출력(소프트맥스, 이진분류) -> 학습(Adam + binary_Crossentropy)
# -> 검증/테스트 평가 -> 시각화

- 말뭉치 로딩(nltk)  데이터 로딩
- 토큰화(빈고 기반 인덱싱) 텍스트를 숫자로 변환
- 시퀀스 패딩   고정길이 배치 구성
- 임베딩  단어를 dense vector표현 학습
- 임베딩  발전
    - 한계 : 작은데이터에서는 일반화 부족
    - 발전 : 사전학습(Word2vec), 문맥적 임베딩(BERT, GPT)

In [5]:
# 토큰화
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# sample data
texts = [
    'I really love this movie',
    'I hate this boring film',
    'love love great film'
]
# 토큰화 객체(최대 단어 10, oov 토큰 지정)
tokenizer = Tokenizer(num_words=10, oov_token='UNK')
tokenizer.fit_on_texts(texts)
print(f'단어인덱스 : {tokenizer.word_index}')
# 시퀀스
seqs = tokenizer.texts_to_sequences(texts)
print(f'원본 시퀀스 : {seqs}')

단어인덱스 : {'UNK': 1, 'love': 2, 'i': 3, 'this': 4, 'film': 5, 'really': 6, 'movie': 7, 'hate': 8, 'boring': 9, 'great': 10}
원본 시퀀스 : [[3, 6, 2, 4, 7], [3, 8, 4, 9, 5], [2, 2, 1, 5]]


defaultdict(int,
            {'this': 2,
             'movie': 1,
             'love': 2,
             'really': 1,
             'i': 2,
             'boring': 1,
             'hate': 1,
             'film': 2,
             'great': 1})